In [27]:
# -*- coding: utf-8 -*-
import argparse
import os
import string
import sys
os.chdir('/data/hyeryung/mucoco')
from itertools import repeat
import torch.multiprocessing as mp
from nb_test_module import Processor, f
from typing import List

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from new_module.utils.robertacustom import RobertaCustomForSequenceClassification

In [29]:
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'


In [30]:
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)

In [31]:
device='cuda'

In [32]:
model = model.to(device)

In [33]:
prompt='abc'
prediction=['dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe']

In [34]:
batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함


In [35]:
output = model(**batch, output_attentions=True, output_hidden_states=True)

In [37]:
attentions = output.attentions
num_layer = 10
max_num_tokens = 6

In [38]:
## 실제 locate하는 로직이 나오는 부분
punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(device)


In [39]:
## attentions : tuple of length num hidden layers
## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
lengths = batch.attention_mask.sum(dim=-1)
# 보고자 하는 attention layer 만 가져옴
attentions = attentions[
    num_layer # originally 10
]
cls_attns = attentions.max(1)[0][:, 0]


In [40]:
## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
## NOTE: softmax를 안했음
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=0.0
no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')
avg_values=cls_attns.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)

In [41]:
## stopwords가 아닌 부분에서만 index를 찾아야 한다.
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
top_masks = (cls_attns >= avg_values.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)

tensor([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False]], device='cuda:0')

In [42]:
max_num_located_tokens = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(device))
max_num_located_tokens = torch.minimum(max_num_located_tokens, top_masks.sum(dim=-1))

In [43]:
top_masks_final = [x[:max_num_located_tokens[i]] for i,x in enumerate(cls_attns.argsort(dim=-1,descending=True).tolist())] 

In [44]:
unit="word"

In [ ]:
if unit == "token":
    locate_ixes=top_masks_final

elif unit == "word":
    proc=Processor(tokenizer)

    try:
        mp.set_start_method('spawn', force=True)
    except RuntimeError:
        pass

    with mp.Pool(3) as pool:
        # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
        locate_ixes = pool.starmap(f, zip(prediction,batch.input_ids.tolist(), lengths.tolist(), top_masks_final,repeat(proc)))

In [32]:
# %%timeit
# [x[:max_num_located_tokens[i]] for i,x in enumerate(cls_attns.argsort(dim=-1,descending=True).tolist())] 
## 이 implementation이 더 빠르다
# 375 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

# from multiprocessing import Pool
# def f(x, length):
#     return x[:length]
# os.environ['TOKENIZERS_PARALLELISM']='true'
# %%timeit

# with Pool(3) as pool:
#     L = pool.starmap(f, zip(cls_attns.argsort(dim=-1,descending=True).tolist(), max_num_located_tokens.tolist()))
# 346 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

375 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'

prompt='abc'
prediction=['dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe']

unit="word"
num_layer = 10
max_num_tokens = 6
device='cuda'

model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)

punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(device)

###
batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함
output = model(**batch, output_attentions=True, output_hidden_states=True)
attentions = output.attentions

## 실제 locate하는 로직이 나오는 부분
## attentions : tuple of length num hidden layers
## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
lengths = batch.attention_mask.sum(dim=-1)
attentions = attentions[num_layer]
cls_attns = attentions.max(1)[0][:, 0] # cls_attns's dimension: (N, L)

## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
## NOTE: softmax를 안했음
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=0.0
no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')
avg_values=cls_attns.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)

## stopwords가 아닌 부분에서만 index를 찾아야 한다.
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
top_masks = (cls_attns >= avg_values.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)
max_num_located_tokens = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(device))
max_num_located_tokens = torch.minimum(max_num_located_tokens, top_masks.sum(dim=-1))
top_masks_final = [x[:max_num_located_tokens[i]] for i,x in enumerate(cls_attns.argsort(dim=-1,descending=True).tolist())] 

if unit == "token":
    locate_ixes=top_masks_final

elif unit == "word":
    proc=Processor(tokenizer)

    try:
        mp.set_start_method('spawn', force=True)
    except RuntimeError:
        pass

    with mp.Pool(3) as pool:
        # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
        locate_ixes = pool.starmap(f, zip(prediction,batch.input_ids.tolist(), lengths.tolist(), top_masks_final,repeat(proc)))

In [ ]:
label_id = 0

## output['hidden_states']: tuple of length num_hidden_layers
## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
layer = output['hidden_states'][0]
layer.retain_grad()

## output['logits'] : (batch_size, num_labels)
softmax=torch.nn.Softmax(dim=-1)
probs = softmax(output['logits'])[:, label_id]

# print(f"probs.shape:{probs.shape}")

probs.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836

## layer.grad : (batch_size, seq_len, hidden_size)
# print(f"layer.grad.shape:{layer.grad.shape}")
norm = torch.norm(layer.grad, dim=-1)

## norm : (batch_size, seq_len)
# print(f"norm.shape:{norm.shape}")
norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
# print(f"norm:{norm}")


In [ ]:
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'

prompt='abc'
prediction=['dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe']

unit="word"
num_layer = 10
max_num_tokens = 6
device='cuda'

model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)

punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(device)

###
batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함
output = model(**batch, output_attentions=True, output_hidden_states=True)
attentions = output.attentions

## 실제 locate하는 로직이 나오는 부분
## attentions : tuple of length num hidden layers
## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
lengths = batch.attention_mask.sum(dim=-1)
attentions = attentions[num_layer]
cls_attns = attentions.max(1)[0][:, 0] # cls_attns's dimension: (N, L)

In [154]:
cls_attns=norm

In [ ]:
class LocateMachine:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def locate_main(self, prediction: List[str], method, max_num_tokens = 6, unit="word", device="cuda", **kwargs):
        
        punctuations = string.punctuation + '\n '
        punctuations = list(punctuations)
        punctuations.remove('-')
        stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in self.tokenizer.special_tokens_map.values()]
        stopwords_ids = self.tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(device)
        batch = self.tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함
        
        if method == "attention":
            output = self.model(**batch, output_attentions=True)
            attentions = output.attentions
            ## attentions : tuple of length num hidden layers
            ## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
            lengths = batch.attention_mask.sum(dim=-1)
            attentions = attentions[kwargs['num_layer']]
            token_wise_scores = attentions.max(1)[0][:, 0] # cls_attns's dimension: (N, L)
        elif method == "grad_norm":
            output = self.model(**batch, output_hidden_states=True)
            ## output['hidden_states']: tuple of length num_hidden_layers
            ## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
            layer = output['hidden_states'][0]
            layer.retain_grad()

            softmax=torch.nn.Softmax(dim=-1)
            probs = softmax(output['logits'])[:, kwargs['label_id']]
            probs.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836
            ## layer.grad : (batch_size, seq_len, hidden_size)
            norm = torch.norm(layer.grad, dim=-1)
            ## norm : (batch_size, seq_len)
            token_wise_scores = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
        
        ## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
        ## NOTE: softmax를 안했음
        token_wise_scores[torch.isin(batch.input_ids, stopwords_ids)]=0.0
        no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')
        avg_values=token_wise_scores.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)

        ## stopwords가 아닌 부분에서만 index를 찾아야 한다.
        token_wise_scores[torch.isin(batch.input_ids, stopwords_ids)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
        top_masks = (token_wise_scores >= avg_values.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)
        max_num_located_tokens = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(device))
        max_num_located_tokens = torch.minimum(max_num_located_tokens, top_masks.sum(dim=-1))
        top_masks_final = [x[:max_num_located_tokens[i]] for i,x in enumerate(token_wise_scores.argsort(dim=-1,descending=True).tolist())] 

        if unit == "token":
            locate_ixes=top_masks_final

        elif unit == "word":
            proc=Processor(self.tokenizer)

            try:
                mp.set_start_method('spawn', force=True)
            except RuntimeError:
                pass

            with mp.Pool(3) as pool:
                # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
                locate_ixes = pool.starmap(f, zip(prediction,batch.input_ids.tolist(), lengths.tolist(), top_masks_final,repeat(proc)))
        
        return locate_ixes

In [156]:
## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
## NOTE: softmax를 안했음
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=0.0
no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')
avg_values=cls_attns.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)

## stopwords가 아닌 부분에서만 index를 찾아야 한다.
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
top_masks = (cls_attns >= avg_values.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)
max_num_located_tokens = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(device))
max_num_located_tokens = torch.minimum(max_num_located_tokens, top_masks.sum(dim=-1))
top_masks_final = [x[:max_num_located_tokens[i]] for i,x in enumerate(cls_attns.argsort(dim=-1,descending=True).tolist())] 

if unit == "token":
    locate_ixes=top_masks_final

elif unit == "word":
    proc=Processor(tokenizer)

    try:
        mp.set_start_method('spawn', force=True)
    except RuntimeError:
        pass

    with mp.Pool(3) as pool:
        # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
        locate_ixes = pool.starmap(f, zip(prediction,batch.input_ids.tolist(), lengths.tolist(), top_masks_final,repeat(proc)))

dsxe<s> [11622, 42336, 0, 1] 3 [0] <nb_test_module.Processor object at 0x7f1724ef3d00>
sdvbfe [28045, 705, 428, 7068] 4 [3] <nb_test_module.Processor object at 0x7f3662327d00>
dsxe<s> [11622, 42336, 0, 1] 3 [0] <nb_test_module.Processor object at 0x7ff7ed375d00>
sdvbfe [28045, 705, 428, 7068] 4 [3] <nb_test_module.Processor object at 0x7f170b936190>
dsxe<s> [11622, 42336, 0, 1] 3 [0] <nb_test_module.Processor object at 0x7f170b9e1eb0>
sdvbfe [28045, 705, 428, 7068] 4 [3] <nb_test_module.Processor object at 0x7f364cd7e190>
dsxe<s> [11622, 42336, 0, 1] 3 [0] <nb_test_module.Processor object at 0x7ff7d3dc8190>
sdvbfe [28045, 705, 428, 7068] 4 [3] <nb_test_module.Processor object at 0x7f170b936190>
dsxe<s> [11622, 42336, 0, 1] 3 [0] <nb_test_module.Processor object at 0x7f364ce29eb0>
sdvbfe [28045, 705, 428, 7068] 4 [3] <nb_test_module.Processor object at 0x7ff7d8067eb0>


In [ ]:

lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
# print(f"lengths: {lengths}")

locate_ixes = []
locate_scores = []
for i in range(batch["input_ids"].shape[0]):
    
    ## norm_ : (seq_len,)
    current_norm = norm[i, :]
    # print(f"norm_ shape: {current_norm.shape}")
    
    ## current_sent : (lengths[i], )
    current_sent = batch["input_ids"][i][: lengths[i]]
    no_punc_indices = torch.where(~torch.isin(current_sent, torch.tensor(stopwords_ids).to(device)))[0]
    
    # print(f"current_sent: {current_sent}")
    # print(f"len(current_sent), lengths[i]: {len(current_sent), lengths[i]}")
    # print(f"no_punc_indices: {no_punc_indices}")
    # print(f"current_sent[no_punc_indices]: {current_sent[no_punc_indices]}")
    # print(f"tokenizer.decode(current_sent[no_punc_indices]): {tokenizer.decode(current_sent[no_punc_indices])}")
    
    ## normalize current_norm
    current_norm = current_norm[: lengths[i]].softmax(-1) 
    # print(f"current_norm after normalizing: {current_norm}")
    
    current_locate_scores = torch.zeros_like(current_norm)
    current_locate_scores[no_punc_indices] = current_norm[no_punc_indices].clone()
    locate_scores.append(current_locate_scores.cpu().detach().tolist())
    
    current_norm = current_norm[no_punc_indices]
    # print(f"current_norm after selecting non stop words indices: {current_norm}")
    
    ## calculate mean value within the sequence
    avg_value = current_norm.view(-1).mean().item()
    # print("avg_value", avg_value)
    
    ## find indices of tokens whose norm value is greater than the mean value
    top_masks = ((current_norm >= avg_value).nonzero().view(-1)) 
    top_masks = top_masks.cpu().tolist()
    torch.cuda.empty_cache()
    # print("indices of non stopwords tokens whose grad norm value is greater than the mean value", top_masks)
    
    ## in case the number of above average gradient norm tokens is greater than the max_num_tokens or 1/3 of the lengths[i]
    if len(top_masks) > min((lengths[i]) // 3, max_num_tokens):
        # print("len(located_tokens) exceeds max_num_tokens or 1/3 of the lengths[i]. Taking top k.")
        top_masks = (
            current_norm.topk(max(min((lengths[i]) // 3, max_num_tokens), 1))[1]
        )
        top_masks = top_masks.cpu().tolist()
        # print("indices of non stopwords tokens located after taking top k", top_masks)
    
    top_masks = no_punc_indices[top_masks].cpu().detach().tolist()
    # print("indices of tokens located", top_masks)
    
    if unit == "token":
        locate_ixes.append(list(set(top_masks)))
    
    elif unit == "word":

        ## group token indices that belong to the same word
        words = tokenizer.decode(current_sent).strip().split()
        word2tok_mapper=Processor(tokenizer)
        # print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
        grouped_tokens = list(word2tok_mapper.get_word2tok(pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})).values())            # j, k = 0, 0
        # grouped_tokens = []
        # grouped_tokens_for_word = []
        # while j < len(current_sent):
        #     if (tokenizer.decode(current_sent[j]).strip() not in stopwords):
        #         while k < len(words):
        #             if tokenizer.decode(current_sent[j]).strip() in words[k]:
        #                 grouped_tokens_for_word.append(j)
        #                 break
        #             else:
        #                 grouped_tokens.append(grouped_tokens_for_word)
        #                 grouped_tokens_for_word = []
        #                 k += 1
        #     j += 1
        # grouped_tokens.append(grouped_tokens_for_word)
        
        ## expand located token indices to include adjacent token indices that belong to the same word as already located tokens
        top_masks.sort()
        top_masks_final = set()
        for index in top_masks:
            if index not in top_masks_final:
                word = [grouped_ixes for grouped_ixes in grouped_tokens if index in grouped_ixes]
                # print("word", word)
                if len(word) > 0:
                    word = set(word[0])
                else:
                    print(f"warning. {index} not in the word groups. decoded value: {tokenizer.decode(index)}")
                    word = set([index])
                top_masks_final |= word
        locate_ixes.append(sorted(list(top_masks_final)))

In [ ]:
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'

prompt='abc'
prediction=['dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe','dsxe<s>','sdvbfe']

unit="word"
num_layer = 10
max_num_tokens = 6
device='cuda'

model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)

punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(device)

###
batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함
output = model(**batch, output_attentions=True, output_hidden_states=True)
attentions = output.attentions

## 실제 locate하는 로직이 나오는 부분
## attentions : tuple of length num hidden layers
## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
lengths = batch.attention_mask.sum(dim=-1)
attentions = attentions[num_layer]
cls_attns = attentions.max(1)[0][:, 0] # cls_attns's dimension: (N, L)

## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
## NOTE: softmax를 안했음
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=0.0
no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')
avg_values=cls_attns.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)

## stopwords가 아닌 부분에서만 index를 찾아야 한다.
cls_attns[torch.isin(batch.input_ids, stopwords_ids)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
top_masks = (cls_attns >= avg_values.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)
max_num_located_tokens = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(device))
max_num_located_tokens = torch.minimum(max_num_located_tokens, top_masks.sum(dim=-1))
top_masks_final = [x[:max_num_located_tokens[i]] for i,x in enumerate(cls_attns.argsort(dim=-1,descending=True).tolist())] 

if unit == "token":
    locate_ixes=top_masks_final

elif unit == "word":
    proc=Processor(tokenizer)

    try:
        mp.set_start_method('spawn', force=True)
    except RuntimeError:
        pass

    with mp.Pool(3) as pool:
        # https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
        locate_ixes = pool.starmap(f, zip(prediction,batch.input_ids.tolist(), lengths.tolist(), top_masks_final,repeat(proc)))

In [ ]:
def locate_grad_norm(prediction, output, tokenizer, batch, max_num_tokens = 6, unit="word", device="cuda", label_id = 1):

    punctuations = string.punctuation + '\n '
    punctuations = list(punctuations)
    punctuations.remove('-')
    stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
    stopwords_ids = [tokenizer.encode(word,add_special_tokens=False)[-1] for word in stopwords]

    ## output['hidden_states']: tuple of length num_hidden_layers
    ## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
    layer = output['hidden_states'][0]
    layer.retain_grad()
    
    ## output['logits'] : (batch_size, num_labels)
    softmax=torch.nn.Softmax(dim=-1)
    probs = softmax(output['logits'])[:, label_id]
    # print(f"probs.shape:{probs.shape}")
    
    probs.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836

    ## layer.grad : (batch_size, seq_len, hidden_size)
    # print(f"layer.grad.shape:{layer.grad.shape}")
    norm = torch.norm(layer.grad, dim=-1)
    ## norm : (batch_size, seq_len)
    # print(f"norm.shape:{norm.shape}")
    norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
    # print(f"norm:{norm}")
    
    lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
    # print(f"lengths: {lengths}")
    
    locate_ixes = []
    locate_scores = []
    for i in range(batch["input_ids"].shape[0]):
        
        ## norm_ : (seq_len,)
        current_norm = norm[i, :]
        # print(f"norm_ shape: {current_norm.shape}")
        
        ## current_sent : (lengths[i], )
        current_sent = batch["input_ids"][i][: lengths[i]]
        no_punc_indices = torch.where(~torch.isin(current_sent, torch.tensor(stopwords_ids).to(device)))[0]
        
        # print(f"current_sent: {current_sent}")
        # print(f"len(current_sent), lengths[i]: {len(current_sent), lengths[i]}")
        # print(f"no_punc_indices: {no_punc_indices}")
        # print(f"current_sent[no_punc_indices]: {current_sent[no_punc_indices]}")
        # print(f"tokenizer.decode(current_sent[no_punc_indices]): {tokenizer.decode(current_sent[no_punc_indices])}")
        
        ## normalize current_norm
        current_norm = current_norm[: lengths[i]].softmax(-1) 
        # print(f"current_norm after normalizing: {current_norm}")
        
        current_locate_scores = torch.zeros_like(current_norm)
        current_locate_scores[no_punc_indices] = current_norm[no_punc_indices].clone()
        locate_scores.append(current_locate_scores.cpu().detach().tolist())
        
        current_norm = current_norm[no_punc_indices]
        # print(f"current_norm after selecting non stop words indices: {current_norm}")
        
        ## calculate mean value within the sequence
        avg_value = current_norm.view(-1).mean().item()
        # print("avg_value", avg_value)
        
        ## find indices of tokens whose norm value is greater than the mean value
        top_masks = ((current_norm >= avg_value).nonzero().view(-1)) 
        top_masks = top_masks.cpu().tolist()
        torch.cuda.empty_cache()
        # print("indices of non stopwords tokens whose grad norm value is greater than the mean value", top_masks)
        
        ## in case the number of above average gradient norm tokens is greater than the max_num_tokens or 1/3 of the lengths[i]
        if len(top_masks) > min((lengths[i]) // 3, max_num_tokens):
            # print("len(located_tokens) exceeds max_num_tokens or 1/3 of the lengths[i]. Taking top k.")
            top_masks = (
                current_norm.topk(max(min((lengths[i]) // 3, max_num_tokens), 1))[1]
            )
            top_masks = top_masks.cpu().tolist()
            # print("indices of non stopwords tokens located after taking top k", top_masks)
        
        top_masks = no_punc_indices[top_masks].cpu().detach().tolist()
        # print("indices of tokens located", top_masks)
        
        if unit == "token":
            locate_ixes.append(list(set(top_masks)))
        
        elif unit == "word":

            ## group token indices that belong to the same word
            words = tokenizer.decode(current_sent).strip().split()
            word2tok_mapper=Processor(tokenizer)
            # print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
            grouped_tokens = list(word2tok_mapper.get_word2tok(pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})).values())            # j, k = 0, 0
            # grouped_tokens = []
            # grouped_tokens_for_word = []
            # while j < len(current_sent):
            #     if (tokenizer.decode(current_sent[j]).strip() not in stopwords):
            #         while k < len(words):
            #             if tokenizer.decode(current_sent[j]).strip() in words[k]:
            #                 grouped_tokens_for_word.append(j)
            #                 break
            #             else:
            #                 grouped_tokens.append(grouped_tokens_for_word)
            #                 grouped_tokens_for_word = []
            #                 k += 1
            #     j += 1
            # grouped_tokens.append(grouped_tokens_for_word)
            
            ## expand located token indices to include adjacent token indices that belong to the same word as already located tokens
            top_masks.sort()
            top_masks_final = set()
            for index in top_masks:
                if index not in top_masks_final:
                    word = [grouped_ixes for grouped_ixes in grouped_tokens if index in grouped_ixes]
                    # print("word", word)
                    if len(word) > 0:
                        word = set(word[0])
                    else:
                        print(f"warning. {index} not in the word groups. decoded value: {tokenizer.decode(index)}")
                        word = set([index])
                    top_masks_final |= word
            locate_ixes.append(sorted(list(top_masks_final)))

            
    return locate_ixes, locate_scores

In [ ]:
   
if method == "attention":
    output = model(**batch, output_attentions=True) # batch여도 처리가능함
    locate_ixes, locate_scores = locate_attn(output.attentions, tokenizer, batch, max_num_tokens, unit, device, kwargs['num_layer'])
elif method == "grad_norm":
    output = model(**batch, output_hidden_states=True) # batch여도 처리가능함
    locate_ixes, locate_scores = locate_grad_norm(output, tokenizer, batch, max_num_tokens, unit, device, kwargs['label_id'])

## assuming batch_size = 1
masked_sequence = batch['input_ids'].clone().detach()
masked_sequence[:, locate_ixes[0]] = tokenizer.mask_token_id ## 이부분에 대해서 처리방식 고민 필요
masked_sequence_text = tokenizer.batch_decode(
    masked_sequence.tolist()
)

In [ ]:
def locate_attn(prediction, attentions, tokenizer, batch, max_num_tokens = 6, unit="word", device="cuda", num_layer=10):

    punctuations = string.punctuation + '\n '
    punctuations = list(punctuations)
    punctuations.remove('-')

    ## attentions : tuple of length num hidden layers
    ## attentions[i] : attention value of ith hidden layer of shape (batch, num_heads, query, value)
    lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
    # 보고자 하는 attention layer 만 가져옴
    attentions = attentions[
        num_layer # originally 10
    ]
    # print(attentions.shape)
    # print(attentions.max(1)[0].shape)
    # print( batch["input_ids"].shape)
    # print( batch["attention_mask"][0,:])
    cls_attns = attentions.max(1)[0][:, 0]
    
    stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
    stopwords_ids = [tokenizer.encode(word,add_special_tokens=False)[-1] for word in stopwords]
    # print("stopwords_ids", torch.tensor(stopwords_ids))

    locate_ixes=[]
    locate_scores = []
    for i, attn in enumerate(cls_attns):
        
        # print("attn.shape", attn.shape)
        current_sent = batch["input_ids"][i][: lengths[i]]
        # print("current_sent", current_sent)
        if device == "cuda":
            no_punc_indices = torch.where(~torch.isin(current_sent, torch.tensor(stopwords_ids).to(torch.device('cuda'))))[0]
        else:
            no_punc_indices = torch.where(~torch.isin(current_sent, torch.tensor(stopwords_ids)))[0]
        # print("no_punc_indices", no_punc_indices)
        # print(f"current_sent[no_punc_indices]: {current_sent[no_punc_indices]}")
        # print(f"tokenizer.decode(current_sent[no_punc_indices]): {tokenizer.decode(current_sent[no_punc_indices])}")
        
        # current tokenizer does not add <s> and </s> to the sentence.
        current_attn = attn[: lengths[i]].softmax(-1) 
        
        current_locate_scores = torch.zeros_like(current_attn)
        current_locate_scores[no_punc_indices] = current_attn[no_punc_indices].clone()
        locate_scores.append(current_locate_scores.cpu().detach().tolist())
        
        # print("current_attn", current_attn)
        current_attn = current_attn[no_punc_indices]
        # print("current_attn", current_attn)
        
        # 이 값의 평균을 구한다.
        avg_value = current_attn.view(-1).mean().item()
        # print("avg_value", avg_value)
        # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다.
        # fixed to reflect that sometimes the sequence length is 1.
        top_masks = ((current_attn >= avg_value).nonzero().view(-1)) 
        torch.cuda.empty_cache()
        top_masks = top_masks.cpu().tolist()
        # print("top_masks", top_masks)
        
        
        # attention 값이 평균보다 큰 토큰의 수가 k개 또는 문장 전체 토큰 수의 1/3 보다 크면  
        if len(top_masks) > min((lengths[i]) // 3, max_num_tokens):
            # 그냥 attention 값 기준 k 개 또는 토큰 수/3 중 작은 수를 뽑는다.
            top_masks = (
                current_attn.topk(max(min((lengths[i]) // 3, max_num_tokens), 1))[1]
            )
            top_masks = top_masks.cpu().tolist()
            # print("top k top_masks", top_masks)
        top_masks_final = no_punc_indices[top_masks]
        # print("top_masks_final", top_masks_final)
        if unit == "token":
            locate_ixes.append(list(set(top_masks_final.cpu().detach().tolist())))
        
        elif unit == "word":
            # word의 일부만 locate 한 경우, word 전체를 locate 한다.
            # 같은 word 안에 있는 token 끼리 묶음.
            words = tokenizer.decode(current_sent).strip().split()
            # print("words", words)
            word2tok_mapper=Processor(tokenizer)
            # print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
            grouped_tokens = list(word2tok_mapper.get_word2tok(pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})).values())
            # j, k = 0, 0
            # grouped_tokens = []
            # grouped_tokens_for_word = []
            # while j < len(current_sent):
            #     if (tokenizer.decode(current_sent[j]).strip() not in stopwords):
            #         # print("tokenizer.decode(current_sent[j])", tokenizer.decode(current_sent[j]))
            #         while k < len(words):
            #             if tokenizer.decode(current_sent[j]).strip() in words[k]:
            #                 grouped_tokens_for_word.append(j)
            #                 break
            #             else:
            #                 grouped_tokens.append(grouped_tokens_for_word)
            #                 grouped_tokens_for_word = []
            #                 k += 1
            #     j += 1
            # grouped_tokens.append(grouped_tokens_for_word)
            # print("grouped_tokens", grouped_tokens)
            
            top_masks_final.sort()
            top_masks_final_final = []
            for index in top_masks_final:
                # print("index", index)
                if index not in top_masks_final_final:
                    word = [grouped_ixes for grouped_ixes in grouped_tokens if index in grouped_ixes]
                    # print("word", word)
                    if len(word) > 0:
                        word = word[0]
                    else:
                        print(f"!!! {index} not in the grouped_ixes {grouped_tokens}")
                        print(f"!!! tokenizer.decode(index): {tokenizer.decode(index)}")
                    top_masks_final_final.extend(word)
            locate_ixes.append(list(set(top_masks_final_final)))

            
    return locate_ixes, locate_scores

def locate_grad_norm(prediction, output, tokenizer, batch, max_num_tokens = 6, unit="word", device="cuda", label_id = 1):

    punctuations = string.punctuation + '\n '
    punctuations = list(punctuations)
    punctuations.remove('-')
    stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
    stopwords_ids = [tokenizer.encode(word,add_special_tokens=False)[-1] for word in stopwords]

    ## output['hidden_states']: tuple of length num_hidden_layers
    ## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
    layer = output['hidden_states'][0]
    layer.retain_grad()
    
    ## output['logits'] : (batch_size, num_labels)
    softmax=torch.nn.Softmax(dim=-1)
    probs = softmax(output['logits'])[:, label_id]
    # print(f"probs.shape:{probs.shape}")
    
    probs.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836

    ## layer.grad : (batch_size, seq_len, hidden_size)
    # print(f"layer.grad.shape:{layer.grad.shape}")
    norm = torch.norm(layer.grad, dim=-1)
    ## norm : (batch_size, seq_len)
    # print(f"norm.shape:{norm.shape}")
    norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
    # print(f"norm:{norm}")
    
    lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
    # print(f"lengths: {lengths}")
    
    locate_ixes = []
    locate_scores = []
    for i in range(batch["input_ids"].shape[0]):
        
        ## norm_ : (seq_len,)
        current_norm = norm[i, :]
        # print(f"norm_ shape: {current_norm.shape}")
        
        ## current_sent : (lengths[i], )
        current_sent = batch["input_ids"][i][: lengths[i]]
        no_punc_indices = torch.where(~torch.isin(current_sent, torch.tensor(stopwords_ids).to(device)))[0]
        
        # print(f"current_sent: {current_sent}")
        # print(f"len(current_sent), lengths[i]: {len(current_sent), lengths[i]}")
        # print(f"no_punc_indices: {no_punc_indices}")
        # print(f"current_sent[no_punc_indices]: {current_sent[no_punc_indices]}")
        # print(f"tokenizer.decode(current_sent[no_punc_indices]): {tokenizer.decode(current_sent[no_punc_indices])}")
        
        ## normalize current_norm
        current_norm = current_norm[: lengths[i]].softmax(-1) 
        # print(f"current_norm after normalizing: {current_norm}")
        
        current_locate_scores = torch.zeros_like(current_norm)
        current_locate_scores[no_punc_indices] = current_norm[no_punc_indices].clone()
        locate_scores.append(current_locate_scores.cpu().detach().tolist())
        
        current_norm = current_norm[no_punc_indices]
        # print(f"current_norm after selecting non stop words indices: {current_norm}")
        
        ## calculate mean value within the sequence
        avg_value = current_norm.view(-1).mean().item()
        # print("avg_value", avg_value)
        
        ## find indices of tokens whose norm value is greater than the mean value
        top_masks = ((current_norm >= avg_value).nonzero().view(-1)) 
        top_masks = top_masks.cpu().tolist()
        torch.cuda.empty_cache()
        # print("indices of non stopwords tokens whose grad norm value is greater than the mean value", top_masks)
        
        ## in case the number of above average gradient norm tokens is greater than the max_num_tokens or 1/3 of the lengths[i]
        if len(top_masks) > min((lengths[i]) // 3, max_num_tokens):
            # print("len(located_tokens) exceeds max_num_tokens or 1/3 of the lengths[i]. Taking top k.")
            top_masks = (
                current_norm.topk(max(min((lengths[i]) // 3, max_num_tokens), 1))[1]
            )
            top_masks = top_masks.cpu().tolist()
            # print("indices of non stopwords tokens located after taking top k", top_masks)
        
        top_masks = no_punc_indices[top_masks].cpu().detach().tolist()
        # print("indices of tokens located", top_masks)
        
        if unit == "token":
            locate_ixes.append(list(set(top_masks)))
        
        elif unit == "word":

            ## group token indices that belong to the same word
            words = tokenizer.decode(current_sent).strip().split()
            word2tok_mapper=Processor(tokenizer)
            # print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
            grouped_tokens = list(word2tok_mapper.get_word2tok(pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})).values())            # j, k = 0, 0
            # grouped_tokens = []
            # grouped_tokens_for_word = []
            # while j < len(current_sent):
            #     if (tokenizer.decode(current_sent[j]).strip() not in stopwords):
            #         while k < len(words):
            #             if tokenizer.decode(current_sent[j]).strip() in words[k]:
            #                 grouped_tokens_for_word.append(j)
            #                 break
            #             else:
            #                 grouped_tokens.append(grouped_tokens_for_word)
            #                 grouped_tokens_for_word = []
            #                 k += 1
            #     j += 1
            # grouped_tokens.append(grouped_tokens_for_word)
            
            ## expand located token indices to include adjacent token indices that belong to the same word as already located tokens
            top_masks.sort()
            top_masks_final = set()
            for index in top_masks:
                if index not in top_masks_final:
                    word = [grouped_ixes for grouped_ixes in grouped_tokens if index in grouped_ixes]
                    # print("word", word)
                    if len(word) > 0:
                        word = set(word[0])
                    else:
                        print(f"warning. {index} not in the word groups. decoded value: {tokenizer.decode(index)}")
                        word = set([index])
                    top_masks_final |= word
            locate_ixes.append(sorted(list(top_masks_final)))

            
    return locate_ixes, locate_scores

def locate_main(prediction: List[str], method, model, tokenizer, max_num_tokens = 6, unit="word", device="cuda", **kwargs): #label_id = 1, num_layer=10):
# def locate_main(prompt, prediction, method, model, tokenizer, max_num_tokens = 6, unit="word", device="cuda", **kwargs): #label_id = 1, num_layer=10):
    
#     ## prompt에 대한 처리를 어떻게 할지 고민이 필요
#     if isinstance(prompt, list) and isinstance(prediction, list):
#         prompt_prediction = [f"{p}{g}" for p, g in zip(prompt, prediction)]
#     else:
#         prompt_prediction = [f"{prompt}{prediction}"]
#     batch = tokenizer(prompt_prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device)
    batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(device) # prediction이 list여도 처리가능함
    
    if method == "attention":
        output = model(**batch, output_attentions=True) # batch여도 처리가능함
        locate_ixes, locate_scores = locate_attn(prediction, output.attentions, tokenizer, batch, max_num_tokens, unit, device, kwargs['num_layer'])
    elif method == "grad_norm":
        output = model(**batch, output_hidden_states=True) # batch여도 처리가능함
        locate_ixes, locate_scores = locate_grad_norm(prediction, output, tokenizer, batch, max_num_tokens, unit, device, kwargs['label_id'])
    
    ## assuming batch_size = 1
    masked_sequence = batch['input_ids'].clone().detach()
    masked_sequence[:, locate_ixes[0]] = tokenizer.mask_token_id ## 이부분에 대해서 처리방식 고민 필요
    masked_sequence_text = tokenizer.batch_decode(
        masked_sequence.tolist()
    )
    return masked_sequence_text

if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description="""Program to locate spans that contributed the most to the prediction of a model\n\
Input format: jsonl or csv with a column named "text" containing the text to be analyzed\n\
Output format: input dataframe with a new column named "located_indices" each of which is a list of indices of tokens. e.g. [2,3,4,8,10]\n\
""")
    parser.add_argument("--method", type=str, choices=["attention","grad_norm"], help="method to use for locating tokens to edit")
    parser.add_argument("--input_path", type=str, help="path to input file")
    parser.add_argument("--output_path", type=str, help="path to output file")
    parser.add_argument("--model_name_or_path", type=str, help="name of model to use or path to the checkpoint to use")
    parser.add_argument("--model_type", type=str, choices=["AutoModelForSequenceClassification", "RobertaCustomForSequenceClassification"], help="name of model to use")
    parser.add_argument("--batch_size", type=int, default=32, help="batch size to use")
    parser.add_argument("--text_column_name", type=str, default="text", help="name of the column containing text for analysis")
    
    args = parser.parse_args()
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    if args.model_type == "AutoModelForSequenceClassification":
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name_or_path)
    elif args.model_type == "RobertaCustomForSequenceClassification":
        model = RobertaCustomForSequenceClassification.from_pretrained(args.model_name_or_path)
    model.to(device)
        
    if args.input_path.endswith(".jsonl"):
        data = pd.read_json(args.input_path, lines=True)
    elif args.input_path.endswith(".csv"):
        data = pd.read_csv(args.input_path)
        
    if "gpt2" in args.input_path: ## unravel data
        print("GPT2 in args.input_path")
        ## unravel the file 
        data['prompt']=data['prompt'].apply(lambda x: x['text'])
        data = data.explode('generations')

        data['text']=data['generations'].apply(lambda x: x['text'])
        data['tokens']=data['generations'].apply(lambda x: x['tokens'])
        data['locate_labels']=data['generations'].apply(lambda x: x.get('locate_labels', np.nan))
        data = data.dropna(subset=['locate_labels'])
        
        del data['generations']
        del data['locate_labels']
        print(data.head())
    
    dataset = Dataset.from_pandas(data)
    if "gpt2" in args.input_path:
        print("GPT2 in args.input_path")
        def collate_fn(batch):
            input_ids = pad_sequence([torch.LongTensor(example["tokens"]) for example in batch], padding_value=tokenizer.pad_token_id, batch_first=True) 
            # print(f"input_ids: {input_ids}")
            batch = {"input_ids": input_ids,
                    "attention_mask": (input_ids != tokenizer.pad_token_id).long()}
            return transformers.tokenization_utils_base.BatchEncoding(batch)
    else:
        def collate_fn(batch):
            batch = tokenizer([example[args.text_column_name] for example in batch], padding=True, truncation=True, return_tensors="pt")
            return batch
    dataloader = DataLoader(dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)
    
    pred_indices = []
    pred_scores = []
    for batch in dataloader:
        batch.to(device)
        results, scores = locate_main(args.method, model, tokenizer, batch, max_num_tokens = 6, num_layer=10, unit="word", use_cuda=True)
        pred_indices.extend(results)
        pred_scores.extend(scores)
    
    data[f'pred_indices_{args.method}'] = pred_indices
    data[f'pred_scores_{args.method}'] = pred_scores
    os.makedirs(os.path.dirname(args.output_path), exist_ok=True)
    data.to_json(args.output_path, lines=True, orient='records')

# Check for discrepancy

In [3]:
#!/usr/bin/env python
# coding: utf-8

from itertools import chain
import math
import argparse
import json
import logging
import os
import time
os.chdir('/data/hyeryung/mucoco')
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer

from copy import deepcopy
import new_module.losses as lossbuilder
import new_module.losses_old as lossbuilder_old
import wandb
# from new_module.decode_utils import (
#     beam_rerank_v0,
#     beam_rerank_v1,
#     beam_rerank_v2,
#     combi_rerank,
# )
# from new_module.new_decode_utils import get_beam_hypotheses, get_combi_hypotheses, final_reranking
from new_module.evaluation.evaluate_wandb import evaluate_main
from new_module.locate.new_locate_utils import LocateMachine
from new_module.locate.locate_utils import locate_main
from new_module.utils.robertacustom import RobertaCustomForSequenceClassification

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))
import joblib
config = joblib.load('config.pkl')

In [4]:
main_start_time = time.time()

if not config.get("model_tag", None):
    if "energy-training" in config["model_paths"][1]:
        config["model_tag"] = "em"
    else:
        config["model_tag"] = "clsf"

    if (config["task"] == "formality") and ("gyafc" in config["model_paths"][1]):
        config["model_tag"] += "-gyafc"

if config["resume"]:
    logger.info("resuming from a previous run")
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        id=config["wandb_run_id"],
        resume="must",
    )
else:
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        config=config,
    )

run_id = run.path.split("/")[-1]
display_name = f"{run_id}"


outdir = os.path.join(config["output_dir_prefix"], display_name)
os.makedirs(outdir, exist_ok=True)
outfile = f"{outdir}/outputs_epsilon{config['min_epsilons'][0]}.txt"
run.summary["outfile_path"] = outfile


class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

## load data
if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    source_dataset = [
        json.loads(l)[config["jsonl_primary_key"]][config["jsonl_secondary_key"]]
        for l in open(config["source_data"])
    ]
    generation_dataset = [
        json.loads(l)["generations"] for l in open(config["source_data"])
    ]
elif (config["task"] == "formality") or (config["task"] == "sentiment-lewis-compr"):
    with open(config["source_data"], "r") as f:
        generation_dataset = [line.rstrip('\n') for line in f.readlines()]
    source_dataset = ["" for l in generation_dataset]

# check if outfile exists
if (config["resume"]) and (os.path.exists(outfile)):

    with open(outfile, "r") as f:
        existing_gens = [x.rstrip("\n") for x in f.readlines()]
    resume_idx = len(existing_gens)
    if resume_idx == len(source_dataset):
        logger.debug("output file is already complete. skipping this run.")
        raise
    elif resume_idx < len(source_dataset):
        logger.info(
            f"output file already exists but is incomplete. resuming from index: {resume_idx}"
        )
        outf = open(outfile, "a")
        int_outf = open(outfile+".intermediate", "a")
    else:
        logger.critical(
            f"output file seems to be corrupted. The file length is {resume_idx}, where the size of source_dataset is {len(source_dataset)}"
        )
        raise
else:
    resume_idx = 0
    outf = open(outfile, "w")
    int_outf = open(outfile+".intermediate", "w")


## load tokenizer, models, define losses
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []

for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].to(config['device'])

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Starting new HTTPS connection (1): api.wandb.ai:443
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 1830
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 377
wandb: Currently logged in as: hayleyson. Use `wandb login --relogin` to force relogin
Popen(['git', 'cat-file', '--batch-check'], cwd=/data/hyeryung/mucoco, stdin=<valid stream>, shell=False, universal_newlines=False)


Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0


50265


https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [5]:

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer


In [6]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)

curr_loss = torch.zeros(len(AR_prediction_all)).to(config['device'])
logging_loss = torch.zeros((len(AR_prediction_all),2)).to(config['device'])
edit_yn = torch.ones(len(AR_prediction_all), dtype=torch.bool).to(config['device'])
        
for lossid, lossname in enumerate(config["losses"]):
    with torch.no_grad():
        lossvalue = lossfns[lossid].compute_gold_loss(
            source_text, AR_prediction_all,
            label_id=config['target_label_ids'][lossid],
        )
        torch.cuda.empty_cache()
    curr_loss += loss_weights[lossid] * lossvalue
    logging_loss[:, lossid] = lossvalue.clone()

In [7]:

# define an object to locate problematic phrases
locator = LocateMachine(lossfns[1].model, lossfns[1].tokenizer)

label_ids = config["target_label_ids"]  # target label's ids for each loss

run.summary["prep_time"] = time.time() - main_start_time
## beginning of main logic
decode_start_time = time.time()
# text_id = 0
if config["resume"]:
    num_skipped = run.summary.get("num_skipped", 0)
    num_edited = run.summary.get("num_edited", 0)
    num_decoded_tokens = run.summary.get("num_decoded_tokens", 0)
else:
    num_skipped = 0
    num_edited = 0
    num_decoded_tokens = 0


In [8]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)


In [9]:
       
running_text = best_text = deepcopy(AR_prediction_all)


In [11]:
running_text = [' wearing games and how ****ing much do I hate horse wearing games.']

In [12]:
masked_text = locator.locate_main(running_text, 
                        method = config['locate_method'], 
                        max_num_tokens = wandb.config.num_edit_token_per_step, 
                        unit = config['locate_unit'], 
                        num_layer = -2, #penultimate
                        label_id = config['target_label_ids'][1])

In [13]:
masked_text


[' wearing<mask> and how<mask><mask> much do I hate<mask> wearing games.']

In [14]:
masked_text_old  = []
for example in running_text:
    masked_text_old_  = locate_main(example, 
                                config["locate_method"], 
                                name2model[config["model_paths"][1]], 
                                name2tokenizer[config["tokenizer_paths"][1]], 
                                max_num_tokens = wandb.config.num_edit_token_per_step, 
                                unit=config["locate_unit"], 
                                device="cuda", 
                                label_id=config["target_label_ids"][1],
                                num_layer=10)
    masked_text_old.append(masked_text_old_[0])

In [15]:
masked_text_old

[' wearing games and how<mask><mask> much do I hate<mask> wearing games.']

In [24]:
masked_text_old == masked_text

True

# Debug for discrepancy

In [23]:
model = name2model[config["model_paths"][1]]
tokenizer = name2tokenizer[config["tokenizer_paths"][1]]

In [19]:
prediction = running_text[0]

In [20]:
prediction

' wearing games and how ****ing much do I hate horse wearing games.'

In [28]:
## old
# def locate_main(prediction, method, model, tokenizer, max_num_tokens = 6, unit="word", device="cuda", **kwargs): #label_id = 1, num_layer=10):
batch_old = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(config['device'])
output_old = model(**batch_old, output_hidden_states=True)


In [181]:
## new
batch_new = tokenizer([prediction], add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(config['device']) # prediction이 list여도 처리가능함
lengths = batch_new.attention_mask.sum(dim=-1)

In [32]:
batch_old

{'input_ids': tensor([[ 2498,   426,     8,   141, 31095,   154,   203,   109,    38,  4157,
          5253,  2498,   426,     4]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [182]:
batch_new

{'input_ids': tensor([[ 2498,   426,     8,   141, 31095,   154,   203,   109,    38,  4157,
          5253,  2498,   426,     4]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
## old locate_gradnorm

In [36]:
## old
# def locate_grad_norm(output, tokenizer, batch, max_num_tokens = 6, unit="word", device="cuda", label_id = 1):
import string
punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = [tokenizer.encode(word,add_special_tokens=False)[-1] for word in stopwords]


In [38]:

## output['hidden_states']: tuple of length num_hidden_layers
## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
layer_old = output_old['hidden_states'][0]
layer_old.retain_grad()


In [39]:
layer_old

tensor([[[ 0.030, -0.345, -0.056,  ...,  0.108,  0.017,  0.061],
         [-0.092,  0.105,  0.213,  ...,  0.420, -0.381,  0.088],
         [ 0.101,  0.066,  0.025,  ..., -0.029, -0.155,  0.407],
         ...,
         [ 0.166, -0.305,  0.111,  ...,  0.303, -0.069, -0.004],
         [ 0.054,  0.213, -0.006,  ...,  0.527, -0.356, -0.161],
         [ 0.519,  0.014, -0.069,  ..., -0.721,  0.081, -0.006]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [41]:
label_id = 1
## output['logits'] : (batch_size, num_labels)
softmax=torch.nn.Softmax(dim=-1)
probs_old = softmax(output_old['logits'])[:, label_id]
# print(f"probs.shape:{probs.shape}")


In [42]:

probs_old.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836


In [43]:
probs_old

tensor([0.439], device='cuda:0', grad_fn=<SelectBackward0>)

In [44]:

## layer.grad : (batch_size, seq_len, hidden_size)
# print(f"layer.grad.shape:{layer.grad.shape}")
norm_old = torch.norm(layer_old.grad, dim=-1)


In [45]:
norm_old

tensor([[0.090, 0.093, 0.069, 0.058, 0.319, 0.092, 0.057, 0.060, 0.040, 0.054,
         0.113, 0.069, 0.061, 0.076]], device='cuda:0')

In [46]:
## norm : (batch_size, seq_len)
# print(f"norm.shape:{norm.shape}")
norm_old = torch.where(norm_old > 0, norm_old, torch.full_like(norm_old, 1e-10))
# print(f"norm:{norm}")


In [47]:
norm_old

tensor([[0.090, 0.093, 0.069, 0.058, 0.319, 0.092, 0.057, 0.060, 0.040, 0.054,
         0.113, 0.069, 0.061, 0.076]], device='cuda:0')

In [48]:

lengths_old = [i.tolist().count(1) for i in batch["attention_mask"]]
# print(f"lengths: {lengths}")


In [49]:
lengths_old

[14]

In [51]:

locate_ixes = []
locate_scores = []
# for i in range(batch["input_ids"].shape[0]):
i = 0

## norm_ : (seq_len,)
current_norm_old = norm_old[i, :]
# print(f"norm_ shape: {current_norm.shape}")


In [52]:
current_norm_old

tensor([0.090, 0.093, 0.069, 0.058, 0.319, 0.092, 0.057, 0.060, 0.040, 0.054,
        0.113, 0.069, 0.061, 0.076], device='cuda:0')

In [53]:
lengths_old

[14]

In [56]:

## current_sent : (lengths[i], )
current_sent_old = batch_old["input_ids"][i][: lengths_old[i]]


In [57]:
current_sent_old

tensor([ 2498,   426,     8,   141, 31095,   154,   203,   109,    38,  4157,
         5253,  2498,   426,     4], device='cuda:0')

In [62]:
for tok in current_sent_old:
    print(tokenizer.decode(tok))

 wearing
 games
 and
 how
 ****
ing
 much
 do
 I
 hate
 horse
 wearing
 games
.


In [65]:
no_punc_indices_old = torch.where(~torch.isin(current_sent_old, torch.tensor(stopwords_ids).to(config['device'])))[0]


In [66]:
no_punc_indices_old

tensor([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], device='cuda:0')

In [63]:

# print(f"current_sent: {current_sent}")
# print(f"len(current_sent), lengths[i]: {len(current_sent), lengths[i]}")
# print(f"no_punc_indices: {no_punc_indices}")
# print(f"current_sent[no_punc_indices]: {current_sent[no_punc_indices]}")
# print(f"tokenizer.decode(current_sent[no_punc_indices]): {tokenizer.decode(current_sent[no_punc_indices])}")

## normalize current_norm
current_norm_old = current_norm_old[: lengths_old[i]].softmax(-1) 
# print(f"current_norm after normalizing: {current_norm}")


In [64]:
current_norm_old

tensor([0.071, 0.072, 0.070, 0.069, 0.090, 0.071, 0.069, 0.069, 0.068, 0.069,
        0.073, 0.070, 0.069, 0.070], device='cuda:0')

In [67]:

current_locate_scores_old = torch.zeros_like(current_norm_old)
current_locate_scores_old[no_punc_indices_old] = current_norm_old[no_punc_indices_old].clone()


In [68]:
current_locate_scores_old

tensor([0.071, 0.072, 0.000, 0.069, 0.090, 0.071, 0.069, 0.069, 0.068, 0.069,
        0.073, 0.070, 0.069, 0.000], device='cuda:0')

In [69]:
locate_scores.append(current_locate_scores_old.cpu().detach().tolist())


In [70]:

current_norm_old = current_norm_old[no_punc_indices]
# print(f"current_norm after selecting non stop words indices: {current_norm}")


In [71]:
current_norm_old

tensor([0.071, 0.072, 0.069, 0.090, 0.071, 0.069, 0.069, 0.068, 0.069, 0.073,
        0.070, 0.069], device='cuda:0')

In [72]:

## calculate mean value within the sequence
avg_value_old = current_norm_old.view(-1).mean().item()
# print("avg_value", avg_value)


In [73]:
avg_value_old

0.07165762782096863

In [74]:

## find indices of tokens whose norm value is greater than the mean value
top_masks_old = ((current_norm_old >= avg_value_old).nonzero().view(-1)) 
top_masks_old = top_masks_old.cpu().tolist()
torch.cuda.empty_cache()
# print("indices of non stopwords tokens whose grad norm value is greater than the mean value", top_masks)


In [75]:
top_masks_old

[3, 9]

In [78]:
(lengths_old[i]) // 3

4

In [77]:
max_num_tokens=5
## in case the number of above average gradient norm tokens is greater than the max_num_tokens or 1/3 of the lengths[i]
if len(top_masks_old) > min((lengths_old[i]) // 3, max_num_tokens):
    # print("len(located_tokens) exceeds max_num_tokens or 1/3 of the lengths[i]. Taking top k.")
    top_masks_old = (
        current_norm.topk(max(min((lengths_old[i]) // 3, max_num_tokens), 1))[1]
    )
    top_masks_old = top_masks_old.cpu().tolist()
    # print("indices of non stopwords tokens located after taking top k", top_masks)

top_masks_old = no_punc_indices[top_masks_old].cpu().detach().tolist()
# print("indices of tokens located", top_masks)


In [79]:
top_masks_old

[4, 10]

In [81]:
from new_module.locate.locate_utils import Processor

In [82]:
## group token indices that belong to the same word
words_old = tokenizer.decode(current_sent_old).strip().split()
word2tok_mapper=Processor(tokenizer)
# print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
grouped_tokens_old = list(word2tok_mapper.get_word2tok(pd.Series({'words':words_old, 'tokens':current_sent_old.cpu().tolist()})).values())            # j, k = 0, 0


In [186]:
grouped_tokens_old

[[0], [1], [2], [3], [4, 5], [6], [7], [8], [9], [10], [11], [12, 13]]

In [187]:
grouped_tokens_new

[[0], [1], [2], [3], [4, 5], [6], [7], [8], [9], [10], [11], [12, 13]]

In [84]:
# grouped_tokens_old = []
# grouped_tokens_old_for_word = []
# while j < len(current_sent):
#     if (tokenizer.decode(current_sent[j]).strip() not in stopwords):
#         while k < len(words):
#             if tokenizer.decode(current_sent[j]).strip() in words[k]:
#                 grouped_tokens_old_for_word.append(j)
#                 break
#             else:
#                 grouped_tokens_old.append(grouped_tokens_old_for_word)
#                 grouped_tokens_old_for_word = []
#                 k += 1
#     j += 1
# grouped_tokens_old.append(grouped_tokens_old_for_word)

## expand located token indices to include adjacent token indices that belong to the same word as already located tokens
top_masks_old.sort()
top_masks_old_final = set()
for index in top_masks_old:
    if index not in top_masks_old_final:
        word = [grouped_ixes for grouped_ixes in grouped_tokens_old if index in grouped_ixes]
        # print("word", word)
        if len(word) > 0:
            word = set(word[0])
        else:
            print(f"warning. {index} not in the word groups. decoded value: {tokenizer.decode(index)}")
            word = set([index])
        top_masks_old_final |= word
locate_ixes.append(sorted(list(top_masks_old_final)))


In [85]:
locate_ixes

[[4, 5, 10]]

In [ ]:
## new

In [122]:
batch_new

{'input_ids': tensor([[ 2498,   426,     8,   141, 31095,   154,   203,   109,    38,  4157,
          5253,  2498,   426,     4]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [123]:
output_new = model(**batch_new, output_hidden_states=True)
## output['hidden_states']: tuple of length num_hidden_layers
## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
layer_new = output_new['hidden_states'][0]
layer_new.retain_grad()


In [89]:
(layer_new != layer_old).sum()

tensor(0, device='cuda:0')

In [124]:

softmax=torch.nn.Softmax(dim=-1)
probs_new = softmax(output_new['logits'])[:, label_id]
probs_new.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836


In [92]:
probs_old

tensor([0.439], device='cuda:0', grad_fn=<SelectBackward0>)

In [91]:
probs_new

tensor([0.439], device='cuda:0', grad_fn=<SelectBackward0>)

In [125]:
## layer.grad : (batch_size, seq_len, hidden_size)
norm_new = torch.norm(layer_new.grad, dim=-1)
## norm : (batch_size, seq_len)
token_wise_scores_new = torch.where(norm_new > 0, norm_new, torch.full_like(norm_new, 1e-10))

In [96]:
norm_old == token_wise_scores_new

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True]], device='cuda:0')

In [126]:
token_wise_scores_new

tensor([[0.090, 0.093, 0.069, 0.058, 0.319, 0.092, 0.057, 0.060, 0.040, 0.054,
         0.113, 0.069, 0.061, 0.076]], device='cuda:0')

In [98]:
punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids_new = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(model.device)


In [99]:
stopwords_ids_new

tensor([    8,     9,    50,    98,   328,   113, 10431,  1629,   207,   947,
          108,  1640,    43,  3226,  2744,     6,     4,    73,    35,   131,
        41552,  5214, 15698,   116,  1039, 10975, 37457,   742, 35227,  1215,
        12905, 45152, 15483, 24303, 34437, 50118,  1437,     0,     2,     3,
            2,     1,     0, 50264], device='cuda:0')

In [108]:
token_wise_scores_new

tensor([[0.090, 0.093, 0.069, 0.058, 0.319, 0.092, 0.057, 0.060, 0.040, 0.054,
         0.113, 0.069, 0.061, 0.076]], device='cuda:0')

In [127]:
## softmax 먼저 해주기 ## <- 이것때문에 결과가 달라졌을 것이다.
token_wise_scores_new[batch_new.attention_mask == 0] = -float("inf")
token_wise_scores_new = token_wise_scores_new.softmax(dim=-1)

In [130]:
current_norm_old

tensor([0.071, 0.072, 0.069, 0.090, 0.071, 0.069, 0.069, 0.068, 0.069, 0.073,
        0.070, 0.069], device='cuda:0')

In [131]:
token_wise_scores_new

tensor([[0.071, 0.072, 0.070, 0.069, 0.090, 0.071, 0.069, 0.069, 0.068, 0.069,
         0.073, 0.070, 0.069, 0.070]], device='cuda:0')

In [134]:
## avg_value만 구하면 된다고 한다면, 이렇게도 가능하다.
token_wise_scores_new[torch.isin(batch_new.input_ids, stopwords_ids_new)]=0.0
no_punc_len_new=((~torch.isin(batch_new.input_ids, stopwords_ids_new))*batch_new.attention_mask).sum(dim=-1) # tensor([2, 4], device='cuda:0')
avg_values_new=token_wise_scores_new.sum(dim=-1)/no_punc_len_new # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)


In [105]:
avg_value_old

0.07165762782096863

In [137]:
avg_values_new[0].item()

0.07165762782096863

In [ ]:

## stopwords가 아닌 부분에서만 index를 찾아야 한다.
token_wise_scores_new[torch.isin(batch_new.input_ids, stopwords_ids_new)]=-float("inf") ## stopwords 인부분을 -inf로 세팅하면, avg랑 비교할 때랑 topk를 찾을때 빠질것으로 예상
top_masks_new = (token_wise_scores_new >= avg_values_new.unsqueeze(1))# unsqueeze to allow implicit broadcasting : (N) -> (N, 1) -> (N, L)
max_num_located_tokens_new = torch.minimum((lengths//3), torch.LongTensor([max_num_tokens]).to(model.device))
max_num_located_tokens_new = torch.minimum(max_num_located_tokens_new, top_masks_new.sum(dim=-1))
top_masks_new_final = [x[:max_num_located_tokens_new[i]] for i,x in enumerate(token_wise_scores_new.argsort(dim=-1,descending=True).tolist())] 


In [145]:
lengths_new = lengths

In [147]:
from new_module.locate.new_locate_utils import get_word_level_locate_indices
from itertools import repeat 
for i, arguments in enumerate(zip(prediction,batch_new.input_ids.tolist(), lengths_new.tolist(), top_masks_new_final, repeat(tokenizer))):
    locate_ixes = get_word_level_locate_indices(*arguments)
    batch_new.input_ids[i, locate_ixes] = tokenizer.mask_token_id
    

In [149]:
locate_ixes

[10, 4]

In [157]:

def get_word2tok(row: pd.Series, tokenizer: AutoTokenizer) -> dict:
    """
    A function that take a list of words and a corresponding list of tokens 
    into a mapping between each word's index and its corresponding token indexes.
    @param row: A row from dataframe
    @return word2char: A dictionary with word's location index as keys and tuples of corresponding token location indexes as values.

    Example:
    row=pd.Series()
    row['words']=['wearing', 'games', 'and', 'holy', '****ing', 'shit', 'do', 'I', 'hate', 'horse', 'wearing', 'games.']
    row['tokens']=[86, 6648, 1830, 290, 11386, 25998, 278, 7510, 466, 314, 5465, 8223, 5762, 1830, 13]
    word2tok=get_word2tok(row)
    word2tok
    {0: [0, 1],
    1: [2],
    2: [3],
    ...
    10: [12],
    11: [13, 14]}
    """
    
    jl, jr, k = 0, 0, 0
    grouped_tokens = []
    tok2word=dict()
    while jr <= len(row['tokens'])+1 and k < len(row['words']):
        
        if tokenizer.decode(row['tokens'][jl:jr]).strip() == row['words'][k]:
            grouped_tokens.append(list(range(jl,jr)))
            for ix in range(jl,jr):
                tok2word[ix] = k
            k += 1
            jl = jr
            jr += 1
        else:
            jr += 1
    # word2tok = dict(zip(range(len(grouped_tokens)), grouped_tokens))
    # return word2tok
    return tok2word, grouped_tokens

def get_word_level_locate_indices(current_sent:str,prediction:list,length:int, top_masks_final:list, tokenizer:AutoTokenizer):
    # word의 일부만 locate 한 경우, word 전체를 locate 한다.
    # 같은 word 안에 있는 token 끼리 묶음.
    words = current_sent.strip().split()
    prediction = prediction[:length]
    tok2word, grouped_tokens = get_word2tok(pd.Series({'words':words, 'tokens':prediction}), tokenizer)
    
    top_masks_final.sort()
    top_masks_final_final = []
    for index in top_masks_final:
        if index not in top_masks_final_final:
            # word = [grouped_ixes for grouped_ixes in grouped_tokens if index in grouped_ixes]
            word_index = tok2word.get(index, None)
            # if len(word) > 0:
            if word_index is not None:
                top_masks_final_final.extend(grouped_tokens[word_index])
            else:
                top_masks_final_final.extend([index])    
    return list(set(top_masks_final_final))

In [150]:

class Processor():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    def get_word2tok(self, row: pd.Series) -> dict:
        """
        A function that take a list of words and a corresponding list of tokens 
        into a mapping between each word's index and its corresponding token indexes.
        @param row: A row from dataframe
        @return word2char: A dictionary with word's location index as keys and tuples of corresponding token location indexes as values.

        Example:
        row=pd.Series()
        row['words']=['wearing', 'games', 'and', 'holy', '****ing', 'shit', 'do', 'I', 'hate', 'horse', 'wearing', 'games.']
        row['tokens']=[86, 6648, 1830, 290, 11386, 25998, 278, 7510, 466, 314, 5465, 8223, 5762, 1830, 13]
        word2tok=get_word2tok(row)
        word2tok
        {0: [0, 1],
        1: [2],
        2: [3],
        ...
        10: [12],
        11: [13, 14]}
        """
        
        jl, jr, k = 0, 0, 0
        grouped_tokens = []
        while jr <= len(row['tokens'])+1 and k < len(row['words']):
            # print(f"{jl}, {jr}, {k}: {self.tokenizer.decode(row['tokens'][jl:jr]).strip()}")
            if self.tokenizer.decode(row['tokens'][jl:jr]).strip() == row['words'][k]:
                grouped_tokens.append(list(range(jl,jr)))
                k += 1
                jl = jr
                jr += 1
            else:
                jr += 1
        word2tok = dict(zip(range(len(grouped_tokens)), grouped_tokens))
        return word2tok

In [175]:
words_old = tokenizer.decode(current_sent_old).strip().split()
word2tok_mapper_old=Processor(tokenizer)
# print(f"input to word2tok: {pd.Series({'words':words, 'tokens':current_sent.cpu().tolist()})}")
grouped_tokens_old = list(word2tok_mapper_old.get_word2tok(pd.Series({'words':words_old, 'tokens':current_sent_old.cpu().tolist()})).values())

In [176]:
grouped_tokens_old

[[0], [1], [2], [3], [4, 5], [6], [7], [8], [9], [10], [11], [12, 13]]

In [183]:
words_new = prediction.strip().split()
tokens_new = batch_new.input_ids.tolist()[0][:lengths[0]] ## batch_new가 마스킹 되기 전 깨끗한 버전이여야 함!
tok2word, grouped_tokens_new = get_word2tok(pd.Series({'words':words_new, 'tokens':tokens_new}), tokenizer)

In [184]:
tok2word

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 11}

Check if debugging worked

In [1]:
#!/usr/bin/env python
# coding: utf-8

from itertools import chain
import math
import argparse
import json
import logging
import os
import time
os.chdir('/data/hyeryung/mucoco')
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer

from copy import deepcopy
import new_module.losses as lossbuilder
import new_module.losses_old as lossbuilder_old
import wandb
# from new_module.decode_utils import (
#     beam_rerank_v0,
#     beam_rerank_v1,
#     beam_rerank_v2,
#     combi_rerank,
# )
# from new_module.new_decode_utils import get_beam_hypotheses, get_combi_hypotheses, final_reranking
from new_module.evaluation.evaluate_wandb import evaluate_main
from new_module.locate.new_locate_utils import LocateMachine
from new_module.locate.locate_utils import locate_main
from new_module.utils.robertacustom import RobertaCustomForSequenceClassification

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))
import joblib
config = joblib.load('config.pkl')

In [2]:
main_start_time = time.time()

if not config.get("model_tag", None):
    if "energy-training" in config["model_paths"][1]:
        config["model_tag"] = "em"
    else:
        config["model_tag"] = "clsf"

    if (config["task"] == "formality") and ("gyafc" in config["model_paths"][1]):
        config["model_tag"] += "-gyafc"

if config["resume"]:
    logger.info("resuming from a previous run")
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        id=config["wandb_run_id"],
        resume="must",
    )
else:
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        config=config,
    )

run_id = run.path.split("/")[-1]
display_name = f"{run_id}"


outdir = os.path.join(config["output_dir_prefix"], display_name)
os.makedirs(outdir, exist_ok=True)
outfile = f"{outdir}/outputs_epsilon{config['min_epsilons'][0]}.txt"
run.summary["outfile_path"] = outfile


class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

## load data
if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    source_dataset = [
        json.loads(l)[config["jsonl_primary_key"]][config["jsonl_secondary_key"]]
        for l in open(config["source_data"])
    ]
    generation_dataset = [
        json.loads(l)["generations"] for l in open(config["source_data"])
    ]
elif (config["task"] == "formality") or (config["task"] == "sentiment-lewis-compr"):
    with open(config["source_data"], "r") as f:
        generation_dataset = [line.rstrip('\n') for line in f.readlines()]
    source_dataset = ["" for l in generation_dataset]

# check if outfile exists
if (config["resume"]) and (os.path.exists(outfile)):

    with open(outfile, "r") as f:
        existing_gens = [x.rstrip("\n") for x in f.readlines()]
    resume_idx = len(existing_gens)
    if resume_idx == len(source_dataset):
        logger.debug("output file is already complete. skipping this run.")
        raise
    elif resume_idx < len(source_dataset):
        logger.info(
            f"output file already exists but is incomplete. resuming from index: {resume_idx}"
        )
        outf = open(outfile, "a")
        int_outf = open(outfile+".intermediate", "a")
    else:
        logger.critical(
            f"output file seems to be corrupted. The file length is {resume_idx}, where the size of source_dataset is {len(source_dataset)}"
        )
        raise
else:
    resume_idx = 0
    outf = open(outfile, "w")
    int_outf = open(outfile+".intermediate", "w")


## load tokenizer, models, define losses
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []

for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].to(config['device'])

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Starting new HTTPS connection (1): api.wandb.ai:443
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 1830
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 377
wandb: Currently logged in as: hayleyson. Use `wandb login --relogin` to force relogin
Popen(['git', 'cat-file', '--batch-check'], cwd=/data/hyeryung/mucoco, stdin=<valid stream>, shell=False, universal_newlines=False)


Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0


50265


https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [3]:

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer


In [4]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)

curr_loss = torch.zeros(len(AR_prediction_all)).to(config['device'])
logging_loss = torch.zeros((len(AR_prediction_all),2)).to(config['device'])
edit_yn = torch.ones(len(AR_prediction_all), dtype=torch.bool).to(config['device'])
        
for lossid, lossname in enumerate(config["losses"]):
    with torch.no_grad():
        lossvalue = lossfns[lossid].compute_gold_loss(
            source_text, AR_prediction_all,
            label_id=config['target_label_ids'][lossid],
        )
        torch.cuda.empty_cache()
    curr_loss += loss_weights[lossid] * lossvalue
    logging_loss[:, lossid] = lossvalue.clone()

In [5]:

# define an object to locate problematic phrases
locator = LocateMachine(lossfns[1].model, lossfns[1].tokenizer)

label_ids = config["target_label_ids"]  # target label's ids for each loss

run.summary["prep_time"] = time.time() - main_start_time
## beginning of main logic
decode_start_time = time.time()
# text_id = 0
if config["resume"]:
    num_skipped = run.summary.get("num_skipped", 0)
    num_edited = run.summary.get("num_edited", 0)
    num_decoded_tokens = run.summary.get("num_decoded_tokens", 0)
else:
    num_skipped = 0
    num_edited = 0
    num_decoded_tokens = 0


In [6]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)


In [7]:
       
running_text = best_text = deepcopy(AR_prediction_all)


In [25]:
config['locate_method'] = 'attention' ## 'grad_norm' ## 두 가지 경우 모두 결과가 동일하게 잘 나온다.

In [26]:
running_text = [' wearing games and how ****ing much do I hate horse wearing games.']

In [27]:
masked_text = locator.locate_main(running_text, 
                        method = config['locate_method'], 
                        max_num_tokens = wandb.config.num_edit_token_per_step, 
                        unit = config['locate_unit'], 
                        num_layer = -2, #penultimate
                        label_id = config['target_label_ids'][1])

In [28]:
masked_text


['<mask> games and how<mask><mask> much do I hate<mask> wearing games.']

In [29]:
masked_text_old  = []
for example in running_text:
    masked_text_old_  = locate_main(example, 
                                config["locate_method"], 
                                name2model[config["model_paths"][1]], 
                                name2tokenizer[config["tokenizer_paths"][1]], 
                                max_num_tokens = wandb.config.num_edit_token_per_step, 
                                unit=config["locate_unit"], 
                                device="cuda", 
                                label_id=config["target_label_ids"][1],
                                num_layer=10)
    masked_text_old.append(masked_text_old_[0])

In [30]:
masked_text_old

['<mask> games and how<mask><mask> much do I hate<mask> wearing games.']

In [31]:
masked_text_old == masked_text

True

In [32]:
       
running_text = best_text = deepcopy(AR_prediction_all)


In [33]:
masked_text = locator.locate_main(running_text, 
                        method = config['locate_method'], 
                        max_num_tokens = wandb.config.num_edit_token_per_step, 
                        unit = config['locate_unit'], 
                        num_layer = -2, #penultimate
                        label_id = config['target_label_ids'][1])

In [34]:
masked_text


['<mask><mask><mask> the majority of<mask> end up in<mask> that you had to drive yourself. My only recourse is to feed it<mask><mask> What would be the happy tale of my life then?',
 '<mask> games and holy ****ing<mask><mask> I hate<mask> wearing games.',
 "<mask><mask><mask> it<mask> makes me want to<mask><mask> every time I see it on the internet, even though it's not worth a thing because I am a<mask><mask>"]

In [35]:
masked_text_old  = []
for example in running_text:
    masked_text_old_  = locate_main(example, 
                                config["locate_method"], 
                                name2model[config["model_paths"][1]], 
                                name2tokenizer[config["tokenizer_paths"][1]], 
                                max_num_tokens = wandb.config.num_edit_token_per_step, 
                                unit=config["locate_unit"], 
                                device="cuda", 
                                label_id=config["target_label_ids"][1],
                                num_layer=10)
    masked_text_old.append(masked_text_old_[0])

In [36]:
masked_text_old

['<mask><mask><mask> the majority of<mask> end up in<mask> that you had to drive yourself. My only recourse is to feed it<mask><mask> What would be the happy tale of my life then?',
 '<mask> games and holy ****ing<mask><mask> I hate<mask> wearing games.',
 "<mask><mask><mask> it<mask> makes me want to<mask><mask> every time I see it on the internet, even though it's not worth a thing because I am a<mask><mask>"]

In [37]:
masked_text_old == masked_text

True

check for detail for logics handling batch with samples with different lengths

In [38]:
running_text

[' dirt. Unfortunately the majority of horses end up in shit that you had to drive yourself. My only recourse is to feed it myself. What would be the happy tale of my life then?',
 ' wearing games and holy ****ing shit do I hate horse wearing games.',
 " fetishes: it just makes me want to puke every time I see it on the internet, even though it's not worth a thing because I am a furry."]

In [55]:
import string
prediction = running_text
tokenizer = lossfns[1].tokenizer
model = lossfns[1].model
device = config['device']
label_id = 1

punctuations = string.punctuation + '\n '
punctuations = list(punctuations)
punctuations.remove('-')
stopwords = [" and", " of", " or", " so"] + punctuations + [token for token in tokenizer.special_tokens_map.values()]
stopwords_ids = tokenizer.batch_encode_plus(stopwords, return_tensors="pt",add_special_tokens=False)['input_ids'].squeeze().to(model.device)


In [44]:
batch = tokenizer(prediction, add_special_tokens=False, padding=True, truncation=True, return_tensors="pt").to(model.device) # prediction이 list여도 처리가능함
lengths = batch.attention_mask.sum(dim=-1)

In [46]:
output = model(**batch, output_hidden_states=True)
## output['hidden_states']: tuple of length num_hidden_layers
## output['hidden_states'][0]: (batch_size, seq_len, hidden_size)
layer = output['hidden_states'][0]
layer.retain_grad()

softmax=torch.nn.Softmax(dim=-1)
probs = softmax(output['logits'])[:,label_id]
probs.sum().backward(retain_graph=True) ## NOTE. https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments/47026836#47026836
## layer.grad : (batch_size, seq_len, hidden_size)
norm = torch.norm(layer.grad, dim=-1)
## norm : (batch_size, seq_len)
token_wise_scores = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))


In [47]:
token_wise_scores

tensor([[    0.029,     0.015,     0.030,     0.014,     0.027,     0.027,
             0.098,     0.037,     0.032,     0.034,     0.156,     0.030,
             0.021,     0.018,     0.018,     0.058,     0.043,     0.021,
             0.013,     0.011,     0.034,     0.014,     0.012,     0.038,
             0.015,     0.022,     0.016,     0.020,     0.009,     0.008,
             0.010,     0.018,     0.019,     0.011,     0.010,     0.018,
             0.009,     0.013],
        [    0.006,     0.005,     0.004,     0.008,     0.006,     0.005,
             0.017,     0.006,     0.003,     0.005,     0.005,     0.006,
             0.004,     0.003,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000],
        [    0.025,     0.020,     0

In [49]:
token_wise_scores_wrong = token_wise_scores.softmax(dim=-1)

In [50]:
token_wise_scores_wrong

tensor([[0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.028, 0.027, 0.026, 0.026,
         0.030, 0.026, 0.026, 0.026, 0.026, 0.027, 0.027, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.027, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026],
        [0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.027, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026],
        [0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.027,
         0.029, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.027, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.030, 0.029, 0.026, 0.026, 0.026, 0.026]],
       device='cuda:0')

In [51]:
token_wise_scores[batch.attention_mask == 0] = -float("inf")
token_wise_scores = token_wise_scores.softmax(dim=-1)

In [52]:
token_wise_scores

tensor([[0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.028, 0.027, 0.026, 0.026,
         0.030, 0.026, 0.026, 0.026, 0.026, 0.027, 0.027, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.027, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026,
         0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026, 0.026],
        [0.071, 0.071, 0.071, 0.072, 0.071, 0.071, 0.072, 0.071, 0.071, 0.071,
         0.071, 0.071, 0.071, 0.071, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000],
        [0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.030,
         0.033, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.030, 0.029,
         0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029, 0.029,
         0.029, 0.030, 0.033, 0.032, 0.000, 0.000, 0.000, 0.000]],
       device='cuda:0')

In [56]:
token_wise_scores[torch.isin(batch.input_ids, stopwords_ids)]=0.0

In [58]:
no_punc_len_wrong=(~torch.isin(batch.input_ids, stopwords_ids)).sum(dim=-1) # tensor([2, 4], device='cuda:0')

In [59]:
no_punc_len_wrong

tensor([32, 12, 31], device='cuda:0')

In [60]:
no_punc_len=(~torch.isin(batch.input_ids, stopwords_ids)*batch.attention_mask).sum(dim=-1) # tensor([2, 4], device='cuda:0')


In [61]:
no_punc_len ## 차이 없음 이유: stopwords에 special tokens들도 들어가서 pad token도 포함되어서 

tensor([32, 12, 31], device='cuda:0')

In [62]:
avg_values=token_wise_scores.sum(dim=-1)/no_punc_len # tensor([0.5120, 0.3744], device='cuda:0', grad_fn=<DivBackward0>)